# Bronze Layer

In this notebook ETL processes of data in bronze layer are develop and tested. 

## Development

In [1]:
# Importar librerias python
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# Importar modulos propios en notebook
project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

In [3]:
from db.connection import engine_setting, engine_connection

In [44]:


rootFolder = Path.cwd().parent

excel_path_1_bonos_trab =  rootFolder / "data" / "U532-2024-11-05.xls"
excel_path_2_taller_orden_rep =  rootFolder / "data" / "U533-2024-11-05.xls"
excel_path_3_bonos_pres =  rootFolder / "data" / "U551-2024-11-05.xls"
excel_path_4_stock =  rootFolder / "data" / "U552-2024-11-05.xls"
excel_path_5_compras =  rootFolder / "data" / "U553-2024-11-05.xls"
excel_path_6_mostrador =  rootFolder / "data" / "U554-2024-11-05.xls"
excel_path_7_invertidas =  rootFolder / "data" / "U555-2024-11-05.xls"
excel_path_8_taller_orden_venta =  rootFolder / "data" / "U560-2024-11-05.xls"
excel_path_9_cliente =  rootFolder / "data" / "U6301303-conta-2024-11-06.xls"
excel_path_10_articulo =  rootFolder / "data" / "U6301303-taller-2024-11-05.xls"
excel_path_11_empresas =  rootFolder / "data" / "U6311303-conta-2024-11-06.xls"
excel_path_12_talleres =  rootFolder / "data" / "U6311303-taller-2024-11-05.xls"
excel_path_13_almacenes =  rootFolder / "data" / "U6321303-2024-11-05.xls"
excel_path_14_operarios =  rootFolder / "data" / "U6331303-2024-11-05.xls"
excel_path_15_vehiculo =  rootFolder / "data" / "U6341303-2024-11-05.xls"
excel_path_16_tipos_horas =  rootFolder / "data" / "ULSTTHPT-2024-11-05.xls"
excel_path_17_tipos_venta =  rootFolder / "data" / "ULSTTVPT-2024-11-05.xls"

In [3]:
rootFolder = Path.cwd().parent

clientes_a = rootFolder/ 'data' / 'U6301303_A_20241202.xls'

df_clientes_a = pd.read_excel(clientes_a)

WARNING *** file size (51826) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [ ]:
# Leer el archivo Excel con la ruta completa
df_bonos_trab = pd.read_excel(excel_path_1_bonos_trab)
df_taller_orden_rep = pd.read_excel(excel_path_2_taller_orden_rep)
df_bono_pres = pd.read_excel(excel_path_3_bonos_pres)
df_stock = pd.read_excel(excel_path_4_stock)
df_compras = pd.read_excel(excel_path_5_compras)
df_mostrador = pd.read_excel(excel_path_6_mostrador)
df_invertidas = pd.read_excel(excel_path_7_invertidas).ffill()
df_taller_orden_venta = pd.read_excel(excel_path_8_taller_orden_venta)
df_cliente = pd.read_excel(excel_path_9_cliente)
df_articulos = pd.read_excel(excel_path_10_articulo)
df_empresas = pd.read_excel(excel_path_11_empresas)
df_talleres = pd.read_excel(excel_path_12_talleres)
df_almacenes = pd.read_excel(excel_path_13_almacenes)
df_operarios = pd.read_excel(excel_path_14_operarios)
df_vehiculos = pd.read_excel(excel_path_15_vehiculo)
df_tipos_horas = pd.read_excel(excel_path_16_tipos_horas)
df_tipos_venta = pd.read_excel(excel_path_17_tipos_venta)



### Conexión con capa bronze

In [ ]:
# = Connection to schema ddbb =

# Define database type and port 
db_type = 'postgresql' 
port = 5432

# Get schema name acording to layer name
db_schema = 'bronze'

# Create engine for the specific schema
engine = engine_setting(db_type=db_type, db_port=port, db_schema=db_schema)

# Establish the connection
conn = engine_connection(engine)

### Funciones de limpieza y transformación de datos.

- Eliminar columnas completamente vacias.
- Eliminar espacios vacios en cada registro.
- Verificar las columns que podrían ser números enteros 11 en lugar de 11.0. 

In [3]:
def all_columns_to_str (df):
    '''
    This function cleans and transforms a DataFrame `df` with the following steps:
    
    1. Drops rows where all columns contain NaN values.
    2. Filters out rows that contain the text '***' in any text column.
    3. Converts float-type columns to integer type (Int64) if all non-NaN values in the column are integers.
    4. Converts all columns to strings (`str`), replacing NaN values with empty strings and removing leading and trailing whitespace.

    Parameters:
    df (pd.DataFrame): The DataFrame to be transformed and cleaned.

    Returns:
    pd.DataFrame: The cleaned and transformed DataFrame.
    '''
    # Drop rows where all columns are NaN
    df.dropna(how='all', inplace=True)

    # Apply the filter to remove rows that contain '***' in any text column
    for column in df.select_dtypes(include='object').columns:
        df = df[~df[column].astype(str).str.contains(r'\*\*\*', na=False)]
    
    # cheach each column of float type to check if it can be integer
    for column in df.select_dtypes(include=["float"]).columns:
        # verifies if each non nan values of the column are integers
        if df[column].dropna().apply(float.is_integer).all():
            # Converts the column to int, keeping NaN values
            df[column] = df[column].astype("Int64")

    for column in df.select_dtypes(include=["object"]).columns:
    # Convert values that can be interpreted as floats to int, keep NaN or non-numeric values
        df[column] = df[column].apply(lambda x: str(x) if pd.isna(x) else int(x) if isinstance(x, float) else x)
            

    
    for column in df.columns:
        # df[column] = df[column].astype(str)
        # df[column] = df[column].replace('nan', '')
        # df[column] = df[column].replace('<NA>', '')
        # df[column] = df[column].replace('NaT', '')
        # df[column] = df[column].str.strip()

        df[column] = df[column].astype(str)
        df[column] = df[column].replace('nan', None)
        df[column] = df[column].replace('<NA>', None)
        df[column] = df[column].replace('NaT', None)
        df[column] = df[column].str.strip()


    return df

In [4]:
df_empresas = all_columns_to_str(df_empresas)
df_almacenes = all_columns_to_str(df_almacenes)
df_operarios = all_columns_to_str(df_operarios)
df_vehiculos = all_columns_to_str(df_vehiculos)
df_tipos_horas = all_columns_to_str(df_tipos_horas)
df_tipos_venta = all_columns_to_str(df_tipos_venta)
df_cliente = all_columns_to_str(df_cliente)
df_articulos = all_columns_to_str(df_articulos)
df_talleres = all_columns_to_str(df_talleres)

df_bonos_trab = all_columns_to_str(df_bonos_trab) 
df_taller_orden_rep = all_columns_to_str(df_taller_orden_rep) 
df_bono_pres = all_columns_to_str(df_bono_pres)
df_stock = all_columns_to_str(df_stock)
df_compras = all_columns_to_str(df_compras)
df_mostrador = all_columns_to_str(df_mostrador)
df_invertidas = all_columns_to_str(df_invertidas)
df_taller_orden_venta = all_columns_to_str(df_taller_orden_venta)

NameError: name 'df_empresas' is not defined

In [ ]:
all_columns_to_str(df_clientes_a)

In [5]:
from processes.bronze.extract import incremental_data

In [6]:
from processes.bronze.extract import incrementales_contabilidad, prefijos_bronze, list_files_in_most_recent_date


In [7]:
mas_recientes_contabilidad, ficheros = list_files_in_most_recent_date(incrementales_contabilidad)

In [8]:
df_cliente = incremental_data(incrementales_contabilidad, mas_recientes_contabilidad, prefijos_bronze["clientes"])

WARNING *** file size (52481) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (14082) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (181436) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (81569) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (12092113) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (2791515) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (1032552) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is 

In [10]:
all_columns_to_str(df_cliente)

,Codigo cuenta,Cat,Nombre cliente,DNI/CIF,Dirección,C.P.,Localidad,E-mail
0,5260,A,HERRERA BALBONTIN BONIFACIO,13740979C,"Bº IGLESIA, 19-A CHALET 5",39192,SAN MAMES DE MERUELO (CANTABRIA),boheba@gmail.com
1,12222,A,BARCENA FERNANDEZ CESAR,13736278B,LA VENTILLA 41,39710,SOLARES (CANTABRIA),cbarcena@lodiscan.es
2,14247,A,"PAZOS DEL RIO,FERNANDO",30557396X,ALTE M VALLECILLA 4 4 CT I,48920,PPORTUGALETE (VIZCAYA),fdopdelrio@gmail.com
3,15225,A,FERREIRA AGUIAR ANTONIO JOSE,72582222H,"ERREBALBURU, 4 4º D",48260,ERMUA-BIZKAIA,None
4,16298,A,PEÑA SAN EMETERIO VICTORINO,13772496G,"RESIDENCIA EL SOL, 2 - CHALET. Nº 8 Bº SOMAVILLA",39766,SAN PANTALEON DE ARAS (CANTABRIA),facturacionvpena@gmail.com
...,...,...,...,...,...,...,...,...
79463,X5006,T,JOSE RAMON SECO,None,None,None,None,None
79464,X5007,T,YOLANDA ALONSO,None,None,None,None,None
79465,X5008,T,LUIS CARLOS,None,None,None,None,None
79466,X5009,T,JOSE BATS OLAGO,None,None,None,None,None


: 

In [9]:
df_cliente['Codigo cuenta']

0         5260.0
1        12222.0
2        14247.0
3        15225.0
4        16298.0
          ...   
79463      X5006
79464      X5007
79465      X5008
79466      X5009
79467      X5010
Name: Codigo cuenta, Length: 79468, dtype: object

In [13]:
df_cliente['Codigo cuenta'].apply(lambda x: str(x) if pd.isna(x) else int(x) if isinstance(x, float) else x)

0         5260
1        12222
2        14247
3        15225
4        16298
         ...  
79463    X5006
79464    X5007
79465    X5008
79466    X5009
79467    X5010
Name: Codigo cuenta, Length: 79393, dtype: object

In [15]:
for value in df_cliente['Codigo cuenta'].head(5):
    try:
        print(str(int(value)))

    except:
        continue

5260
12222
14247
15225
16298


In [ ]:
df_cliente['Codigo cuenta'].head(5)

## Data upload - Master tables

### Update new data

**Step 1**: Get existing values of the reference column in ddbb table.

**Step 2**: Compare dataframe values with ddbb existing values.

**Step 3**: If there are new values, values are going to be written in th bbdd.

**Considerations**
- All handle values need to be str type.
- Clean white spaces for all values.


In [48]:
# One function 

def upload_new_data_master (table_name, df, key_column, engine):
    '''
    Process new values: 
    - Get existing values from the database.
    - Identify new values in the dataframe.
    - Write new values to the database table.

    input: 
        table_name (str): name of the table in the database.
        df (DataFrame): dataframe to be compared with existing values.
        key_column (str): key column name in the dataframe and table database to compare.
        engine (SQLAlchemy engine): database connection.

    output:
        None
    '''
    # Get existing values from the database
    query = f'SELECT "{key_column}" FROM "{table_name}"'
    existing_values = pd.read_sql(query, con=engine)[key_column].tolist()

    # Get new data by comparing dataframe with existing values
    df_new_data = df[~df[key_column].isin(existing_values)]

    # Write new values to the database
    if not df_new_data.empty:
        df_new_data.to_sql(table_name, con=engine, if_exists='append', index=False)


### Update modified data

In [49]:
# Load data form database

def get_database_table (table_name):

    query = f'SELECT * FROM "{table_name}"'
    db_table = pd.read_sql(query, con=engine)

    return (db_table)

Update modified data

In [ ]:
# simulación 

df_empresas_mod = df_empresas.copy()
df_empresas_mod.loc[df_empresas_mod['Codig'] == '13', 'Nombre'] = 'Nombre Empresa Modificado'
df_empresas_mod

In [ ]:
df_empresas_mod

In [ ]:
merge_data = df_empresas_mod.merge(
        df_empresas, on='Codig',
        suffixes=('_new', '_db')
    )
merge_data

In [ ]:
# Filter and make sure both DataFrames have the same column names and index
new_merge_data = merge_data.filter(regex='_new$').sort_index(axis=1)
db_data = merge_data.filter(regex='_db$').sort_index(axis=1)

# Otherwise column names are adjusted
new_merge_data.columns = new_merge_data.columns.str.replace('_new$', '', regex=True)
db_data.columns = db_data.columns.str.replace('_db$', '', regex=True)

# Compare rows where there are differences
modified_data = merge_data[(new_merge_data != db_data).any(axis=1)]

modified_data

## Data update - Fact tables

In [52]:
from datetime import date, timedelta

def upload_new_data_fact (table_name, df, key_column, date_column, engine):
    '''
    Process new values: 
    - Get existing values from the database.
    - Identify new values in the dataframe.
    - Write new values to the database table.

    input: 
        table_name (str): name of the table in the database.
        df (DataFrame): dataframe to be compared with existing values.
        key_column (str): key column name in the dataframe and table database to compare.
        date_column (str): date column name in the dataframe and table database to compare.
        engine (SQLAlchemy engine): database connection.

    output:
        None
    '''
    # get date range
    current_date = date.today()
    min_date = current_date - timedelta(days=120)

    current_date = current_date.strftime('%Y-%m-%d')
    min_date = min_date.strftime('%Y-%m-%d') # pd.to_datetime(df[date_column], format='%Y-%m-%d', errors='coerce').min().strftime('%Y-%m-%d')
    
    # Get existing values from the database
    query = f'''
        SELECT "{key_column}", "{date_column}"
        FROM "{table_name}" 
        WHERE "{date_column}" BETWEEN '{min_date}' AND '{current_date}'
        '''
    existing_values = pd.read_sql(query, con=engine)

    # Get new data by comparing dataframe with existing values

    # Generate combined keys for comparison to identify new data
    df = df.assign(
        combined_key=df[key_column].astype(str) + "_" + df[date_column].astype(str)
    )
    existing_values = existing_values.assign(
        combined_key=existing_values[key_column].astype(str) + "_" + existing_values[date_column].astype(str)
    )

    # Filter rows in df that are not in existing_values
    df_new_data = df[~df['combined_key'].isin(existing_values['combined_key'])]

    # Drop the temporary 'combined_key' column before saving new values
    df_new_data = df_new_data.drop(columns='combined_key')
    existing_values = existing_values.drop(columns='combined_key')

    # Write new values to the database
    if not df_new_data.empty:
        df_new_data.to_sql(table_name, con=engine, if_exists='append', index=False)

    # Optional: return df_new_data if needed for further processing or testing
    # return df_new_data

In [ ]:
len(df_taller_orden_rep['REF.OR'].unique())

In [ ]:
df_taller_orden_rep['REF.OR'].value_counts()

In [ ]:
df_taller_orden_rep[df_taller_orden_rep['REF.OR'] == '']

In [ ]:
df_taller_orden_rep.loc[1180: 1190, :]

In [21]:
# Insert existing data sample
df_taller_orden_rep_sample =  df_taller_orden_rep.loc[:15, :]
upload_new_data_fact ('Taller OrdenReparacion U533', df_taller_orden_rep_sample, 'REF.OR', 'Fec.apertu', engine)

In [22]:
# upload new data sample
df_taller_orden_rep_new_data = df_taller_orden_rep.loc[:25, :]
upload_new_data_fact ('Taller OrdenReparacion U533', df_taller_orden_rep_new_data, 'REF.OR', 'Fec.apertu', engine)

Validación Tabla Ordenes Venta Mostrador

In [11]:
df_mostrador_sample = df_mostrador.loc[:15, :]
df_mostrador_new_data = df_mostrador.loc[:25, :]

In [14]:
upload_new_data_fact('Taller OrdenVentaMostrador U554', df_mostrador_sample, 'Refer.', 'Fecha', engine)

In [15]:
upload_new_data_fact('Taller OrdenVentaMostrador U554', df_mostrador_new_data, 'Refer.', 'Fecha', engine)

Validación Tabla Ordenes Venta taller

In [26]:
df_taller_ord_venta_sample = df_taller_orden_venta.loc[:14, :]
df_taller_ord_venta_new_data = df_taller_orden_venta.loc[:24, :]

In [28]:
upload_new_data_fact('Taller OrdenVentaTaller U560', df_taller_ord_venta_sample, 'Refer.', 'Fec.sali', engine)

In [29]:
upload_new_data_fact('Taller OrdenVentaTaller U560', df_taller_ord_venta_new_data, 'Refer.', 'Fec.sali', engine)

Validación Tabla Stock - No tiene key column 

In [30]:
df_stock_sample = df_stock.loc[:14, :]
df_stock_new_data = df_stock.loc[:24, :]

In [ ]:
upload_new_data_fact('Taller OrdenVentaTaller U560', df_taller_ord_venta_sample, 'Refer.', 'Fec.sali', engine)

In [ ]:
df_stock['Artículo'].value_counts().sort_values(ascending=True)

In [ ]:
df_stock[df_stock['Artículo'] == '3/CA679']

Validación de Tabla Compras - agregar otra key_column

In [40]:
df_compras_sample = df_compras.loc[:14, :]
df_compras_new_data = df_compras.loc[:24, :]

In [41]:
upload_new_data_fact('Compras U553', df_compras_sample, 'Referenci', 'Fecha', engine)

In [42]:
upload_new_data_fact('Compras U553', df_compras_new_data, 'Referenci', 'Fecha', engine)

Validación de Tabla Bonos Presencia 

In [53]:
def upload_new_data_fact_mult (table_name, df, key_columns, date_column, engine):
    '''
    Process new values: 
    - Get existing values from the database.
    - Identify new values in the dataframe.
    - Write new values to the database table.

    input: 
        table_name (str): name of the table in the database.
        df (DataFrame): dataframe to be compared with existing values.
        key_columns (list): list ok key column names in the dataframe and table database to compare.
        date_colum (str): date column name in the dataframe and table database to compare.
        engine (SQLAlchemy engine): database connection.

    output:
        None
    '''
    # get date range
    current_date = date.today()
    min_date = current_date - timedelta(days=120)

    current_date = current_date.strftime('%Y-%m-%d')
    min_date = min_date.strftime('%Y-%m-%d') # pd.to_datetime(df[date_column], format='%Y-%m-%d', errors='coerce').min().strftime('%Y-%m-%d')
    
    # Get existing values from the database
    query = f'''
        SELECT "{key_columns[0]}", "{key_columns[1]}", "{date_column}"
        FROM "{table_name}" 
        WHERE "{date_column}" BETWEEN '{min_date}' AND '{current_date}'
        '''
    existing_values = pd.read_sql(query, con=engine)#[key_column].tolist()

    # Get new data by comparing dataframe with existing values
    # Generate combined keys for comparison to identify new data
    #df.loc[:, 'combined_key'] = df[key_columns[0]].astype(str) + "_" + df[key_columns[1]].astype(str) + "_" + df[date_column].astype(str)
    #existing_values.loc[:, 'combined_key'] = existing_values[key_columns[0]].astype(str) + "_" + existing_values[key_columns[1]].astype(str) + "_" + existing_values[date_column].astype(str)

    # Crear 'combined_key' en df
    df = df.assign(
        combined_key=df[key_columns[0]].astype(str) + "_" + 
                    df[key_columns[1]].astype(str) + "_" + 
                    df[date_column].astype(str)
    )

    # Crear 'combined_key' en existing_values
    existing_values = existing_values.assign(
        combined_key=existing_values[key_columns[0]].astype(str) + "_" + 
                    existing_values[key_columns[1]].astype(str) + "_" + 
                    existing_values[date_column].astype(str)
    )

    # Filter rows in df that are not in existing_values
    df_new_data = df[~df['combined_key'].isin(existing_values['combined_key'])]

    # Drop the temporary 'combined_key' column before saving new values
    df_new_data = df_new_data.drop(columns='combined_key')
    existing_values = existing_values.drop(columns='combined_key')

    # Write new values to the database
    if not df_new_data.empty:
        df_new_data = df_new_data.copy().sort_values(date_column, ascending=True, inplace=True)
        df_new_data.to_sql(table_name, con=engine, if_exists='append', index=False)

    # Optional: return df_new_data if needed for further processing or testing
    # return df_new_data

Validación Tabla Bonos Presencia

In [54]:
df_bono_pres_sample = df_bono_pres.loc[:15, :]
df_bono_pres_new_data = df_bono_pres.loc[:24, :]

In [39]:
upload_new_data_fact_mult('U551_Presencia', df_bono_pres_sample, ['Operario', 'ENTRADA PRES'], 'Fecha', engine)

In [40]:
upload_new_data_fact_mult('U551_Presencia', df_bono_pres_new_data, ['Operario', 'ENTRADA PRES'], 'Fecha', engine)

In [ ]:
df_bono_pres_sample.sort_values('Fecha', ascending=True)


In [ ]:
df_bono_pres_new_data

Validación Tabla Bonos Trabajadas

In [34]:
df_bonos_trab_sample = df_bonos_trab.loc[:15, :]
df_bonos_trab_new_data = df_bonos_trab.loc[:25, :]

In [36]:
upload_new_data_fact_mult('BonosTrabajadas U532', df_bonos_trab_sample, ['Operario', 'Entra'], 'Fecha', engine)

In [39]:
upload_new_data_fact_mult('BonosTrabajadas U532', df_bonos_trab_new_data, ['Operario', 'Entra'], 'Fecha', engine)

Validación Tabla Compras

In [40]:
df_compras_sample = df_compras.loc[:15, :]
df_compras_new_data = df_compras.loc[:25, :]

In [42]:
upload_new_data_fact_mult('Compras U553', df_compras_sample, ['Referenci', 'Artículo'], 'Fecha', engine)

In [45]:
upload_new_data_fact_mult('Compras U553', df_compras_new_data, ['Referenci', 'Artículo'], 'Fecha', engine)

Validación Tabla invertidas

In [46]:
df_invertidas_sample = df_invertidas.loc[:15, :]
df_invertidas_new_data = df_invertidas.loc[:25, :]

In [49]:
upload_new_data_fact_mult('Invertidas U555', df_invertidas_sample, ['REF.OR', 'Operario'], 'Fec.apertu', engine)

In [50]:
upload_new_data_fact_mult('Invertidas U555', df_invertidas_new_data, ['REF.OR', 'Operario'], 'Fec.apertu', engine)

Validación Tabla Stock 

In [51]:
df_stock_sample = df_stock.loc[:15, :]
df_stock_new_data = df_stock.loc[:25, :]

In [54]:
upload_new_data_fact_mult('Stock U552', df_stock_sample, ['Alma', 'Ref.articulo'], 'F.ul.ent', engine)

In [55]:
upload_new_data_fact_mult('Stock U552', df_stock_new_data, ['Alma', 'Ref.articulo'], 'F.ul.ent', engine)

In [56]:
if conn:
    conn.close()


## Test

Create historic and new data tests dataframes. 

Tabla bonos presencia

In [ ]:
df_bono_pres.sort_values('Fecha', ascending=True)

In [24]:
# Proporción del tamaño de la muestra (ejemplo: 70% para df1, 30% para df2)
tamano_muestra_1 = int(len(df_bono_pres) * 0.7)

# División del DataFrame
df1 = df_bono_pres.iloc[:tamano_muestra_1]  # Primera parte
df2 = df_bono_pres.iloc[-tamano_muestra_1:]  # Segunda parte (con intersección)

# Uniendo algunos datos repetidos (puede ajustarse al criterio)
df_comun = df_bono_pres.iloc[tamano_muestra_1//2:tamano_muestra_1]

# Agregar datos comunes
df1 = pd.concat([df1, df_comun]).drop_duplicates().reset_index(drop=True)
df2 = pd.concat([df2, df_comun]).drop_duplicates().reset_index(drop=True)

# print("Primera muestra (df1):")
# print(df1)

# print("\nSegunda muestra (df2):")
# print(df2)
